In [1]:
import h5py
import tensorflow as tf
import numpy as np
from numpy import f2py

import time

import eos
import eos_t
import universe
import util

# TensorFlow

In [2]:
#tf.debugging.set_log_device_placement(True)

## New `spectrum.gmlt_spec_od_pwc_exact` implementation 

We try to keep a $N^2$ tensor, where each row corresponds to a Doppler profile. At the end, we sum over its vertical axis to get the cumulative profile.

In [4]:
# demonstrate tf.reduce_sum
x = np.reshape(np.arange(16), (4,4))
print('x:\n', x)
print('x summed over vertical axis:', tf.reduce_sum(x, axis=0))

print()

# demonstrate broadcasting
diff = np.reshape(np.arange(4), (4,1))
print('x - diff:\n', x - diff)

x:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]
x summed over vertical axis: tf.Tensor([24 28 32 36], shape=(4,), dtype=int64)

x - diff:
 [[ 0  1  2  3]
 [ 3  4  5  6]
 [ 6  7  8  9]
 [ 9 10 11 12]]


[Rotate a tensor row-wise](https://stackoverflow.com/questions/58173427/how-to-rotate-a-tensor-row-wise-in-tensorflow)

In [77]:
# gather_nd doesn't mess with the gradient!

# with tf.GradientTape() as tape:
#     var = tf.Variable(2.0, dtype='float64')
#     A = tf.Variable([[4, 0, 0],
#               [1, 2, 3],
#               [0, 0, 5]], dtype='float64')
#     A = tf.multiply(A, var)
#     B = tf.gather_nd(A, [[1,2]])

# print('B:', B)
# print('dB/dvar:', tape.gradient(B, var))

[Roll rows of a matrix independently](https://stackoverflow.com/questions/20360675/roll-rows-of-a-matrix-independently) (NOTE: this solution messes with the gradient, so I don't implement it)

In [70]:
# this solution doesn't work; indexing creates a gradient of None

# with tf.GradientTape() as tape:
#     tape.watch(var)
#     tape.watch(A)
#     tape.watch(B)
    
#     var = tf.Variable(2.0, dtype='float64')
#     A = tf.Variable([[4, 0, 0],
#               [1, 2, 3],
#               [0, 0, 5]], dtype='float64')
#     A = tf.multiply(A, var)
#     B = tf.Variable(A[...])

# print(tape.gradient(B, var))

# roll rows independently (NOTE: this messes with the gradient)
# A = np.array([[4, 0, 0],
#               [1, 2, 3],
#               [0, 0, 5]])
# r = np.array([2, 0, -1])

# rows, column_indices = np.ogrid[:A.shape[0], :A.shape[1]]

# r = r % A.shape[1]
# # shifting the "origin" to the right by r is the same as shifting it to the left by (n - r)
# column_indices = column_indices - r[:, np.newaxis]
# # if the first row of column_indices is [-1, 0, 1], then x = [a, b, c] is rolled to 
# # x_r = [x[-1], x[0], x[1]] = [c, a, b]

# result = A[rows, column_indices]
# print('rolled array:\n', result)

In [69]:
#print('testing linspace:\n', tf.linspace([0., 5.], [10., 40.], 5, axis=-1))

# # demonstrate diag and inv
# diag = tf.linalg.diag(np.arange(4) + 1)
# diag = tf.cast(diag, dtype='float64')
# tf.linalg.inv(diag)

# # test tf.pad
# paddings = tf.constant([[0, 0], [0, 6 - 4]])
# tf.pad(x, paddings, "CONSTANT", constant_values=0)

### pix_locations (a potential alternative to tf.roll and ipixes)

pix_locations would be a possible replacement for tf.roll and ipixes. It would have shape (N, N), with each row containing a profile's "relevant" index values located at those indices, e.g., one row could be [0, 0, 2, 3, 4, 0, ..., 0] or [4, 0, 2, 3]. In particular, it would allow different profiles to have different pixel window widths.

### implementation 1 of pix_locations (complete)

This implementation uses a for-loop in two places. In the future, these might be replaced with (or without) vectorized_map in order to improve performance.

In [166]:
# demonstrate scatter add
x = tf.Variable(x)

ind_list = [[1,0], [2, 1], [3, 2]]
updates = [-100, -100, -100]
new_x = tf.tensor_scatter_nd_add(x, ind_list, updates)
print('new_x:', new_x)

# get a list of lists
iterations = tf.constant([2, 3, 2])
row_nums_repeated = [[i]*((int) (iterations[i])) for i in range(3)]
print('list of lists:', row_nums_repeated)

new_x: tf.Tensor(
[[   0    0    0    0    0    0    0    0    0    0]
 [-100    0    0    0    0    0    0    0    0    0]
 [   0 -100    0    0    0    0    0    0    0    0]
 [   0    0 -100    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]], shape=(10, 10), dtype=int32)
list of lists: [[0, 0], [1, 1, 1], [2, 2]]


In [163]:
# complete implementation (with 2 for-loops)

n = 10
x = tf.zeros([10, 10], dtype='int32')
new_x = tf.identity(x)

ipix_hi = tf.constant([2, 7, 11])
ipix_lo = tf.constant([-3,5,8])
profile_widths = ipix_hi - ipix_lo + 1

# ragged tensor; the ith row is range(ipix_lo[i], ipix_hi[i] + 1)
ipixes = tf.ragged.range(ipix_lo, ipix_hi + 1)

# row_nums would go from 0 through N-1
iterations = tf.identity(profile_widths)
row_nums_repeated = [[i]*((int) (iterations[i])) for i in range(3)]
row_nums_repeated = tf.ragged.constant(row_nums_repeated)
#row_nums_repeated = tf.ragged.constant([[0, 0], [1, 1, 1], [2, 2]])

# get the "relevant" index locations
ind_list = tf.stack((row_nums_repeated, ipixes), axis=2)
ind_list = ind_list % n # index wrapping
print('ind_list:', ind_list.to_list())

for i in range(3):
    new_x = tf.tensor_scatter_nd_update(new_x, ind_list[i], ipixes[i])
print('new_x:', new_x)

ind_list: [[[0, 7], [0, 8], [0, 9], [0, 0], [0, 1], [0, 2]], [[1, 5], [1, 6], [1, 7]], [[2, 8], [2, 9], [2, 0], [2, 1]]]
new_x: tf.Tensor(
[[ 0  1  2  0  0  0  0 -3 -2 -1]
 [ 0  0  0  0  0  5  6  7  0  0]
 [10 11  0  0  0  0  0  0  8  9]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]], shape=(10, 10), dtype=int32)


### implementation 2 of pix_locations (incomplete/flawed)

This implementation is flawed. Currently, it doesn't handle indices that are out-of-bounds (e.g. 1024).

In [132]:
# use a mask to get the "relevant" indices
n = 4

ipix_hi = tf.constant([3, 3, 5])
ipix_lo = tf.constant([0,1,2])

pix_locations = tf.repeat([tf.range(n)], n, axis=0)

mask = tf.logical_and(tf.range(n) >= ipix_lo[:, None],
                      tf.range(n) <= ipix_hi[:, None])
mask

<tf.Tensor: shape=(3, 10), dtype=bool, numpy=
array([[False, False, False, False, False, False,  True,  True, False,
        False],
       [False, False, False, False, False,  True,  True,  True, False,
        False],
       [False, False, False,  True,  True, False, False, False, False,
        False]])>

[Assigning to slices of a 2D array](https://stackoverflow.com/questions/48876162/assigning-to-slices-of-2d-numpy-array)

In [127]:
arr = tf.Variable([[1,2,3,4],
                [1,2,3,4],
                [1,2,3,4],
                [1,2,3,4]])

idxs = np.array([0,1,2,0])

mask = tf.range(4) > idxs[:,None]
#print(mask)
arr = tf.multiply(tf.cast(mask, dtype='int32'), arr)
#print(arr)

tf.Tensor(
[[False  True  True  True]
 [False False  True  True]
 [False False False  True]
 [False  True  True  True]], shape=(4, 4), dtype=bool)
tf.Tensor(
[[0 2 3 4]
 [0 0 3 4]
 [0 0 0 4]
 [0 2 3 4]], shape=(4, 4), dtype=int32)


## Defining a function and its derivative
[Custom gradients](https://www.tensorflow.org/guide/eager#custom_gradients)

In [3]:
@tf.custom_gradient
def bar(x, y): # f(x,y) = x * y
    def grad(upstream): # pass in the "upstream" gradient
        dz_dx = y
        dz_dy = x
        return upstream * dz_dx, upstream * dz_dy # pass grad "downstream"
    z = x * y
    return z, grad

x = tf.constant(2.0, dtype=tf.float32)
y = tf.constant(3.0, dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    tape.watch(y)
    z = bar(x, y)
    
print('z:', z)
print('dz/dx:', tape.gradient(z, x))
print('dz/dy:', tape.gradient(z, y))

z: tf.Tensor(6.0, shape=(), dtype=float32)
dz/dx: tf.Tensor(3.0, shape=(), dtype=float32)
dz/dy: tf.Tensor(2.0, shape=(), dtype=float32)


## Vectorization with gmlt_spec_od_pwc_exact

### Outer loop (going through each cell in the LOS)

In [4]:
n_array = tf.constant((3, 6, 0, 7), dtype=tf.float64)

# assert all(i > 0 for i in n_array), "n_array should only have positive values"

# only look at cells where n > 0
inds = np.arange(tf.size(n_array).numpy())
inds = inds[(n_array > 0).numpy()]
inds

array([0, 1, 3])

### Inner loop (adding the Doppler profile to the tau skewer)

In [5]:
l = 10 # LOS length; same as num_pixels and num_elements

def ind_wrap(i):
    '''
    Performs util.gmlt_index_wrap on an index, where r (in this case, l) is 
    pre-specified outside the scope of this method.
    
    PARAMETERS
    ----------
    i: index (an int tensor)
    
    '''
    
    return util.gmlt_index_wrap(i, l)

In [6]:
ipixes = tf.constant(range(6,13), dtype=tf.int32)
jw = tf.map_fn(ind_wrap, ipixes)
jw = tf.reshape(jw, [tf.size(jw).numpy(), 1]) # necessary for tensor_scatter_nd_add

v_domain = tf.constant(1.0e5, dtype=tf.float64) # fake data
pixel_dv = v_domain / l

ipixes = tf.cast(ipixes, dtype=tf.float64)
v_pixel = tf.math.multiply(pixel_dv, tf.math.add(ipixes,0.5))
#jw = tf.map_fn(test_ind_wrap, ipixes)

# more fake data
vlc_l = pixel_dv * 6
vlc_h = pixel_dv * 7

v_doppler = 1.0e4
dxl = (v_pixel - vlc_l) / v_doppler
dxh = (v_pixel - vlc_h) / v_doppler

n_array = tf.cast(tf.ones([l]) * 1e-21, dtype=tf.float64)
tau_array = tf.cast(tf.zeros([l]), dtype=tf.float64)

# calculate tau
tau_i = tf.math.multiply(n_array[6], (tf.math.erf(dxl) - tf.math.erf(dxh)))
tau_array = tf.tensor_scatter_nd_add(tau_array, jw, tau_i)

print(jw)
print(tau_i)
print(tau_array)

tf.Tensor(
[[6]
 [7]
 [8]
 [9]
 [0]
 [1]
 [2]], shape=(7, 1), dtype=int32)
tf.Tensor(
[1.04099972e-21 4.45605255e-22 3.34879004e-23 4.06208906e-25
 7.42901733e-28 1.96608723e-31 7.32747173e-36], shape=(7,), dtype=float64)
tf.Tensor(
[7.42901733e-28 1.96608723e-31 7.32747173e-36 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.04099972e-21 4.45605255e-22
 3.34879004e-23 4.06208906e-25], shape=(10,), dtype=float64)


## Vectorization with gmlt_spec_od_grid

### outline
- arr has shape (x,y,z)
- reshape arr to (x*y, z)
- vectorized_map(fn, arr) returns arr2, shape (x*y, z)
    - here, fn is gmlt_spec_od_pwc_exact
- reshape arr2 to (x,y,z)
- create od_pwc_exact2: only arg is a tuple of the 3 grids, while snapshot-specific args (e.g. redshift) are specified outside the function
    - define this within gmlt_spec_od_grid

In [7]:
def fn(a, factor):
    return a*factor

fact = 2
def fn2(a):
    return fn(a, fact)

# testing tf.reshape

arr = tf.constant(np.arange(24).reshape(2,3,4))
#print(arr)

# we want to reshape arr to [[0,1,2,3], ..., [20,21,22,23]]
arr = tf.reshape(arr, (6, 4))

# arr2 = tf.vectorized_map(fn2, arr)
arr2 = tf.map_fn(fn2, arr, fn_output_signature=tf.int64)
print(arr2)
print(tf.reshape(arr2, (2,3,4)))

tf.Tensor(
[[ 0  2  4  6]
 [ 8 10 12 14]
 [16 18 20 22]
 [24 26 28 30]
 [32 34 36 38]
 [40 42 44 46]], shape=(6, 4), dtype=int64)
tf.Tensor(
[[[ 0  2  4  6]
  [ 8 10 12 14]
  [16 18 20 22]]

 [[24 26 28 30]
  [32 34 36 38]
  [40 42 44 46]]], shape=(2, 3, 4), dtype=int64)


In [8]:
@tf.function()
def outer_product(a):
    return tf.tensordot(a, a, 0)

batch_size = 10
a = tf.ones((batch_size, 4, 4))
c = tf.vectorized_map(outer_product, a)
c.shape

TensorShape([10, 4, 4, 4, 4])

## Vectorization with nyx_eos

In [9]:
eos_obj = eos.EOS_at_z(2.99)

# nyx_eos can accept tensors of different dtypes without errors
t1 = tf.Variable([2], dtype=tf.int32)
t2 = tf.constant([2], dtype=tf.float64)
eos_obj.nyx_eos_vec((t1, t2))

9.087522466956245e+23

In [10]:
x = tf.Variable(np.arange(100).reshape(10,10), dtype=tf.double)
y = tf.Variable(np.arange(100).reshape(10,10), dtype=tf.double)

size = tf.size(x).numpy()
#elems=(x,y)
elems = (tf.reshape(x, [size]), tf.reshape(y, [size]))

# vectorized_map throws a "[Tensor] can't be converted to double" error
#tf.vectorized_map(eos_obj.nyx_eos_vec, elems)
tf.map_fn(eos_obj.nyx_eos_vec, elems, fn_output_signature=tf.float64)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([0.00000000e+00, 4.54376123e+23, 9.08752247e+23, 1.36312837e+24,
       1.81750449e+24, 2.27188062e+24, 2.72625674e+24, 3.18063286e+24,
       3.63500899e+24, 4.08938511e+24, 4.54376123e+24, 4.99813736e+24,
       5.45251348e+24, 5.90688960e+24, 6.36126573e+24, 6.81564185e+24,
       7.27001797e+24, 7.72439410e+24, 8.17877022e+24, 8.63314634e+24,
       9.08752247e+24, 9.54189859e+24, 9.99627471e+24, 1.04506508e+25,
       1.09050270e+25, 1.13594031e+25, 1.18137792e+25, 1.22681553e+25,
       1.27225315e+25, 1.31769076e+25, 1.36312837e+25, 1.40856598e+25,
       1.45400359e+25, 1.49944121e+25, 1.54487882e+25, 1.59031643e+25,
       1.63575404e+25, 1.68119166e+25, 1.72662927e+25, 1.77206688e+25,
       1.81750449e+25, 1.86294211e+25, 1.90837972e+25, 1.95381733e+25,
       1.99925494e+25, 2.04469256e+25, 2.09013017e+25, 2.13556778e+25,
       2.18100539e+25, 2.22644300e+25, 2.27188062e+25, 2.31731823e+25,
       2.36275584e+25, 2.4081

## Vectorization example

In [11]:
# a function with specified parameter types can accept tf.Tensors
def fn2(a: float, b: float):
    return a + b

In [15]:
# elems is a set of tensors, c is a number
def fn(elems, c=3):
    tf.compat.v1.enable_eager_execution()
    print('eager:', tf.executing_eagerly()) # still False

    # convert_to_tensor doesn't fix it
    a = tf.convert_to_tensor(elems[0], dtype=tf.float32, name='test')
    b = elems[1]
    
    print('a:', a)
    print('type(a):', type(a))
    print('eager:', tf.executing_eagerly())
    
    # nyx_eos does NOT accept tf.Tensors
    # (when using vectorized_map, a is a tf.Tensor)
    print(eos_obj.nyx_eos_vec((3,a)))
    
    #print(fn2(a,b))
    
    return a + b + c

Getting the value of a tf.Tensor: https://stackoverflow.com/questions/33633370/how-to-print-the-value-of-a-tensor-object-in-tensorflow

In [16]:
x = tf.constant(np.arange(100).reshape(10,10), dtype=tf.float32)
y = tf.constant(np.arange(100).reshape(10,10), dtype=tf.float32)
elems = (x,y)

start = time.time()
print('eager:', tf.executing_eagerly())
out = tf.vectorized_map(fn, elems=elems)

#time_stats(time.time() - start, 10)

eager: True
eager: False
a: Tensor("loop_body/GatherV2:0", shape=(10,), dtype=float32)
type(a): <class 'tensorflow.python.framework.ops.Tensor'>
eager: False


StagingError: in user code:

    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:188 f  *
        iters,
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:248 _pfor_impl  **
        loop_fn_outputs = loop_fn(loop_var)
    /global/homes/j/jupiter/.conda/envs/lya-tf/lib/python3.7/site-packages/tensorflow/python/ops/parallel_for/control_flow_ops.py:472 loop_fn
        return fn(gathered_elems)
    <ipython-input-15-5c3bb74a1f94>:16 fn
        print(eos_obj.nyx_eos_vec((3,a)))
    /global/u2/j/jupiter/lya-tf/lya_fields/eos.py:49 nyx_eos_vec
        return eos_t.eos.nyx_eos(self.z, rhob, temp)

    SystemError: eos_t.eos.nyx_eos() 3rd argument (t) can't be converted to double


In [ ]:
out

## Assigning an element to an existing tensor

In [17]:
x = tf.zeros([3,3,3])
#tf.tensor_scatter_nd_add?

In [18]:
skewer = tf.zeros([3])
for i in range(3):
    for j in range(3):
        x = tf.tensor_scatter_nd_add(x, [[i,j]], [skewer])
        skewer = skewer + 1
x

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [4., 4., 4.],
        [5., 5., 5.]],

       [[6., 6., 6.],
        [7., 7., 7.],
        [8., 8., 8.]]], dtype=float32)>

In [19]:
x = tf.zeros([3])
tf.tensor_scatter_nd_add(x, [[1]], [3])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 3., 0.], dtype=float32)>

## Testing speed of tensor access

It takes too long to access a tensor's entries one-by-one; I need to avoid this when I'm passing fields into the EOS or optical depth routines

In [20]:
def time_stats(duration, n):
    rate = duration / n**2
    print("Duration:", np.round(duration, 4))
    print("Hours needed to access 1024^3 entries:", 
      np.round(rate * 1024**3 / 3600, 4))

In [21]:
n = 5
field = tf.zeros((n,n))
total = 0

start = time.time()

for i in range(n):
    for j in range(n):
        total += field[i,j]

duration = time.time() - start
time_stats(duration, n)

Duration: 0.0072
Hours needed to access 1024^3 entries: 86.2777


### Nested loops

In [22]:
count = 15
n = 10
field = tf.zeros((n,n))

start = time.time()

# outer loop
i = tf.constant(0)
condition1 = lambda i, count: tf.less(i, n)

def body1(i, r): # index i and result r
    # inner loop
    j = tf.constant(0)
    condition2 = lambda i,j,r: tf.less(j, n)
    
    def body2(i, j, r):
        r += field[i,j]
        return i, tf.add(j, 1), r
    
    i, j, r = tf.while_loop(condition2, body2, loop_vars=[i,j, r])
    
    # increment r
    return tf.add(i, 1), r

# do the loop:
r = tf.while_loop(condition1, body1, [i, count])
print(r)

duration = time.time() - start
time_stats(duration, n)

(<tf.Tensor: shape=(), dtype=int32, numpy=10>, <tf.Tensor: shape=(), dtype=float32, numpy=15.0>)
Duration: 0.0294
Hours needed to access 1024^3 entries: 87.5833


## Testing tf gradients

In [23]:
tf.math.pow(z, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=36.0>

In [24]:
z = tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(z) # without this line, grad is None
    a = tf.divide(1, z+1)
    print('a:', a)
    
grad = tape.gradient(a, z)
print('grad:', grad)

a: tf.Tensor(0.25, shape=(), dtype=float32)
grad: tf.Tensor(-0.0625, shape=(), dtype=float32)


# Wrapping Fortran functions to Python

[Three ways to wrap](https://numpy.org/doc/stable/f2py/f2py.getting-started.html)

- Quick way (this requires Python 3.7 and NumPy 1.18): `python3 -m numpy.f2py -c eos-t.f90 -m eos_t`


In [25]:
from platform import python_version
print(python_version())

3.7.10


In [26]:
eos_t.atomic_rates.tabulate_rates()
print(eos_t.atomic_rates.alphahp)

[1.26271073e-10 1.25558019e-10 1.24848791e-10 ... 6.57445059e-19
 6.47458444e-19 6.37622769e-19]


# Load in the data

In [4]:
filename = "../../../../../cscratch1/sd/jupiter/sim2_z3_FGPA_cgs.h5"

## Testing classes

In [5]:
import snapshot
import pickle

snap = snapshot.Snapshot(filename)

In [10]:
### test snapshot methods
start = time.time()

## while on the Cori GPU node, loading a full 1024^3 field throws an error 
# (possibly due to GPU memory being full)
#temp = snap.read_field('/native_fields/temperature')
#rhob = snap.read_field('/native_fields/baryon_density')
# print(temp.shape)
# print(temp.size)
# print('Time:', time.time() - start)

In [27]:
# test read_field2 (reading in n^3 grids)
start = time.time()
shape = [1,1024,1024]

# loading in 2 1x1024^2 fields went okay, but 10*1024^2 threw an error
rhob2 = snap.read_subfield('/native_fields/baryon_density', shape)
temp2 = snap.read_subfield('/native_fields/temperature', shape)
print(temp2.shape)
print(temp2.size)
print('Time:', time.time() - start)

[1, 1024, 1024]
tf.Tensor([2.05993652e-02 2.10937500e+01 2.10937500e+01], shape=(3,), dtype=float64)
Time: 10.777246952056885


### universe

In [10]:
u = snap.universe
chi = 10
z = 1
u.chi_to_proper_cgs(chi, z)

<tf.Tensor: shape=(), dtype=float64, numpy=2.2856888888888885e+25>

In [6]:
u.h

<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.675>

### Saving the universe and eos_obj objects (see [this link](https://stackoverflow.com/questions/4529815/saving-an-object-data-persistence))

In [21]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [11]:
save_object(u, 'universe.pkl')

with open('universe.pkl', 'rb') as inp:
    univ2 = pickle.load(inp)

univ2.h

<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.675>

In [11]:
omega_b = u.omega_b
h = u.h

rho_crit_100_cgs = 1.8788200386793017e-29
mean_rhob_cgs = omega_b * h*h * rho_crit_100_cgs
z = 2.9999991588912964
a = 1 / (z + 1)
a3_inv = 1.0 / (a * a * a)
rhob_cgs_conversion = mean_rhob_cgs * a3_inv

eos_obj = eos.EOS_at_z(z, rhob_cgs_conversion)

In [22]:
save_object(eos_obj, 'eos_obj.pkl')

### eos

In [19]:
import eos

eos_obj = eos.EOS_at_z(snap.z, rhob_cgs_conversion)
# returns nhi
#eos_obj.nyx_eos(rhob2.field[0,0,0] * 1e-29, temp2.field[0,0,0])
eos_obj.nyx_eos(1e-29, 4)

1.485004133494724e-09

## Metadata

In [17]:
## test the "in" keyword
snap = h5py.File(filename,'r')

name = 'aux_fields'

snap.keys()

if name in snap:
    print('ok')
    
snap.close()

shape = snap['domain'].attrs['shape']
size = snap['domain'].attrs['size']

z = snap['universe'].attrs['redshift']
omega_b = snap['universe'].attrs['omega_b']
omega_m = snap['universe'].attrs['omega_m']
omega_l = snap['universe'].attrs['omega_l']
h = snap['universe'].attrs['hubble']

scale_factor = 1.0 / (1.0 + z)

In [ ]:
print(shape)
print(size)

In [ ]:
snap.close()